# Quick start

In [11]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.20.0


In [15]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [16]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [28]:
class MyModel(Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [29]:
model.build((None, 28, 28, 1))

C:\Users\aksha\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\layer.py:421: UserWarning: `build()` was called on layer 'my_model_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [30]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [31]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [32]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [33]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [34]:
EPOCHS = 5

for epoch in range(EPOCHS):
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result():0.2f}, '
    f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
    f'Test Loss: {test_loss.result():0.2f}, '
    f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
  )

Epoch 1, Loss: 0.14, Accuracy: 95.83, Test Loss: 0.07, Test Accuracy: 97.63
Epoch 2, Loss: 0.04, Accuracy: 98.65, Test Loss: 0.05, Test Accuracy: 98.29
Epoch 3, Loss: 0.02, Accuracy: 99.22, Test Loss: 0.06, Test Accuracy: 98.12
Epoch 4, Loss: 0.01, Accuracy: 99.53, Test Loss: 0.06, Test Accuracy: 98.48
Epoch 5, Loss: 0.01, Accuracy: 99.64, Test Loss: 0.07, Test Accuracy: 98.23


# Customization

In [35]:
print(tf.math.add(1,2))
print(tf.math.add([1,2], [3,4]))
print(tf.math.square(5))
print(tf.math.reduce_sum([1,2,3]))

print(tf.math.square(2) + tf.math.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [36]:
x = tf.linalg.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


In [38]:
import numpy as np

ndarray = np.ones([3,3])

tensor = tf.math.multiply(ndarray, 42)
print(tensor)

print(np.add(tensor, 1))

print(tensor.numpy())

tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## Custom Layers

In [39]:
print(tf.config.list_physical_devices('GPU'))

[]


In [42]:
layer = tf.keras.layers.Dense(100)

inputs = tf.keras.Input(shape=(5,))
layer = tf.keras.layers.Dense(10)(inputs)

In [45]:
layer = tf.keras.layers.Dense(10)
layer(tf.zeros([10, 5]))

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [46]:
layer.variables

[<Variable path=dense_14/kernel, shape=(5, 10), dtype=float32, value=[[ 0.09493351  0.1367799   0.4647326   0.6270594   0.34561384  0.27413708
   -0.4416899  -0.6197559  -0.55538     0.2285018 ]
  [ 0.01951963  0.4286757  -0.5985536  -0.00546265 -0.5825936  -0.07357699
    0.4641536   0.10085022  0.099307    0.13832343]
  [ 0.00316191  0.29172814  0.11536306  0.22098708  0.41280085  0.5044802
   -0.36440912  0.5363887   0.4450119  -0.23176336]
  [ 0.6034524   0.32484388 -0.5823196   0.16472149 -0.19801208  0.3349766
   -0.5122793  -0.30728507  0.26741385  0.4522348 ]
  [ 0.25584286 -0.6188108   0.2023868  -0.5536311  -0.40183574 -0.02214581
    0.14596826 -0.432675   -0.3387089   0.3898347 ]]>,
 <Variable path=dense_14/bias, shape=(10,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]>]

In [48]:
layer.kernel

<Variable path=dense_14/kernel, shape=(5, 10), dtype=float32, value=[[ 0.09493351  0.1367799   0.4647326   0.6270594   0.34561384  0.27413708
  -0.4416899  -0.6197559  -0.55538     0.2285018 ]
 [ 0.01951963  0.4286757  -0.5985536  -0.00546265 -0.5825936  -0.07357699
   0.4641536   0.10085022  0.099307    0.13832343]
 [ 0.00316191  0.29172814  0.11536306  0.22098708  0.41280085  0.5044802
  -0.36440912  0.5363887   0.4450119  -0.23176336]
 [ 0.6034524   0.32484388 -0.5823196   0.16472149 -0.19801208  0.3349766
  -0.5122793  -0.30728507  0.26741385  0.4522348 ]
 [ 0.25584286 -0.6188108   0.2023868  -0.5536311  -0.40183574 -0.02214581
   0.14596826 -0.432675   -0.3387089   0.3898347 ]]>

In [49]:
layer.bias

<Variable path=dense_14/bias, shape=(10,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]>

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = self.add_weight('kernel',
                                     shape=[int(input_shape[-1]),
                                           self.num_outputs])

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

_ = layer(tf.zeros([10,5]))